### Introduction & Notebook Objectives

### Setup & Install Dependencies

In [ ]:
#Install Dependencies
!pip install pandas numpy scikit-learn nltk matplotlib seaborn

In [17]:
#pull from get
!git fetch origin
!git reset --hard origin/main

From https://github.com/Sorrel-Alvason/cs-ticket-categorizer
   cb6e5ee..6e8590a  main       -> origin/main


HEAD is now at 6e8590a Add files via upload


In [19]:
#push to git
!git status
!git add 01_data_preprocessing.ipynb
!git commit -m "Edited Data Preprocessing Notebook"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   01_data_preprocessing.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


[main bf585d4] Edited Data Preprocessing Notebook
 1 file changed, 170 insertions(+), 7 deletions(-)


To https://github.com/Sorrel-Alvason/cs-ticket-categorizer.git
   6e8590a..bf585d4  main -> main


### Load the Dataset

#### Overview
In this Section, we will: 
    Load the dataset from 'dataset-tickets-milti-lang-4-20k.csv'
    Display the first few rows to understand its structure. 
    Filter out the necessary columns ('subject', 'body', 'queue', 'priority')
    check for missing values and basic statistics. 

#### Dataset Description
The dataset consists of customer service tickets with the following columns:
- **`subject`**: The subject line of the ticket (short summary).
- **`body`**: The detailed content of the ticket.
- **`queue`**: The category or type of issue the ticket belongs to.
- **`priority`**: The urgency level of the ticket.

These will be the key inputs (**features**) and outputs (**labels**) for our machine learning model.

In [21]:
import pandas as pd

# Load dataset
df = pd.read_csv("../data/dataset-tickets-multi-lang-4-20k.csv")

# Display basic info
print("Dataset Shape:", df.shape)
print("Column Names:", df.columns.tolist())

# Select only relevant columns
df = df[['subject', 'body', 'queue', 'priority']]

# Show first few rows
df.head()

Dataset Shape: (20000, 15)
Column Names: ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8']


,subject,body,queue,priority
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,Die Datenanalyse-Plattform brach unerwartet ab...,General Inquiry,low
1,Customer Support Inquiry,Seeking information on digital strategies that...,Customer Service,medium
2,Data Analytics for Investment,I am contacting you to request information on ...,Customer Service,medium
3,Krankenhaus-Dienstleistung-Problem,Ein Medien-Daten-Sperrverhalten trat aufgrund ...,Customer Service,high
4,Security,"Dear Customer Support, I am reaching out to in...",Customer Service,medium


### Explore and Understand Data

#### 3. Inspect the Dataset

Before proceeding with preprocessing, we need to understand:
- The total number of entries (**shape of the dataset**).
- The available columns (**column names**).
- Sample rows (**`df.head()`**).
- How many missing values are present (**`df.isnull().sum()`**).

This helps us determine if any cleaning is necessary before feature engineering.


In [23]:
# Check for missing values
df.isnull().sum()

# Basic statistics for text length (subject and body)
df['subject_length'] = df['subject'].apply(lambda x: len(str(x)))
df['body_length'] = df['body'].apply(lambda x: len(str(x)))

df[['subject_length', 'body_length']].describe()

,subject_length,body_length
count,20000.000000,20000.000000
mean,41.813750,394.438400
std,21.170191,248.260435
min,3.000000,3.000000
25%,31.000000,198.000000
50%,41.000000,357.000000
75%,53.000000,555.000000
max,412.000000,2259.000000


### Clean and Preprocess Text Data

### Tokenization

### Stemming/lemmatization

### Saving the Cleaned Dataset